In [36]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
from run_sequence_labeling_crf_labert import FLAGS, SequenceLabelingProcessor, model_fn_builder,modeling,tokenization_labert,file_based_convert_examples_to_features,file_based_input_fn_builder
import tensorflow as tf
import modeling

In [67]:
init_checkpoint = '/home/ops/jupyter/语义/dataprocess/摘要-标题-图谱/内容理解/Lattice_Bert/tmp/best/'


In [3]:
# tf.compat.v1.app.flags.DEFINE_string('f', '', 'kernel')


In [114]:
import pickle


In [226]:
base_dir = '/home/ops/jupyter/语义/dataprocess/摘要-标题-图谱/内容理解/Lattice_Bert/chinese_labert-lite-std-512/'
FLAGS.labert_config_file= base_dir + 'labert_config.json'
FLAGS.vocab_file = base_dir + 'vocab.txt'
FLAGS.lexicon_file = base_dir + 'lexicon.txt'
FLAGS.label_file = base_dir + '../tmp/labels.pkl'
FLAGS.init_checkpoint = init_checkpoint
labert_config = modeling.BertConfig.from_json_file(FLAGS.labert_config_file)
label_list = pickle.load(open(FLAGS.label_file, 'rb'))
processor = SequenceLabelingProcessor()
processor.set_labels(label_list)
pos_indices = [index for index, label in enumerate(label_list) if label.lower() != 'o']


num_train_steps = None
num_warmup_steps = None
save_checkpoints_steps = 1000


tokenizer = tokenization_labert.LatticeTokenizer(
      vocab_file=FLAGS.vocab_file,
      lexicon_file=FLAGS.lexicon_file,
      do_lower_case=FLAGS.do_lower_case)

is_per_host = tf.compat.v1.estimator.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.compat.v1.estimator.tpu.RunConfig(
                master=None,
                # model_dir=init_checkpoint,
                save_checkpoints_steps=save_checkpoints_steps,
                keep_checkpoint_max=1,
                log_step_count_steps=1 << 25,
                tpu_config=tf.compat.v1.estimator.tpu.TPUConfig(
                  iterations_per_loop=FLAGS.iterations_per_loop,
                  num_shards=8,
                  per_host_input_for_training=is_per_host))

model_fn = model_fn_builder(
    labert_config=labert_config,
    num_labels=len(label_list),
    init_checkpoint=False,
    learning_rate=FLAGS.learning_rate,
    beta1=FLAGS.adam_beta1,
    beta2=FLAGS.adam_beta2,
    epsilon=FLAGS.adam_epsilon,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    lr_layer_decay_rate=FLAGS.lr_layer_decay_rate,
    pos_indices=pos_indices)

    # If TPU is not available, this will fall back to normal Estimator on CPU
    # or GPU.
estimator = tf.compat.v1.estimator.tpu.TPUEstimator(
          model_dir = init_checkpoint,
          use_tpu=False,
          model_fn=model_fn,
          config=run_config,
          predict_batch_size=FLAGS.predict_batch_size)

print(1)

INFO:tensorflow:number of lexicon entries: 79145
INFO:tensorflow:number of vocab entries: 101985
INFO:tensorflow:Using config: {'_model_dir': '/home/ops/jupyter/语义/dataprocess/摘要-标题-图谱/内容理解/Lattice_Bert/tmp/best/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_t

In [227]:
contents_predict(estimator, test_file, out_file)

INFO:tensorflow:Writing example 0 of 2
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-0
INFO:tensorflow:tokens: [CLS] 美 纹 纸 真 的 让 人 又 爱 又 恨 ！ 真的 [SEP]
INFO:tensorflow:input_ids: 101 5400 5291 5290 4695 4637 6374 781 1347 4262 1347 2615 8012 21234 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [228]:
result

<generator object TPUEstimator.predict at 0x7f58e0570200>

In [229]:
test_file = '/home/ops/jupyter/语义/dataprocess/摘要-标题-图谱/内容理解/Lattice_Bert/predict/test.txt'
out_file = os.path.dirname(test_file) + '/pre.txt'
def contents_predict(estimator,test_file ,out_file):
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
    tpu_cluster_resolver = None

    predict_examples = processor._create_examples(processor.get_raw_data(test_file), "test")
    # predict_examples = processor.get_test_examples(test_file)
    num_actual_predict_examples = len(predict_examples)
    predict_file = os.path.join(os.path.dirname(test_file), "predict.tf_record")
    file_based_convert_examples_to_features(
      predict_examples, label_list, FLAGS.max_seq_length, tokenizer, predict_file)

    tf.compat.v1.logging.info("***** Running prediction *****")
    tf.compat.v1.logging.info("  Num examples = %d (%d actual, %d padding)",
                              len(predict_examples), num_actual_predict_examples,
                              len(predict_examples) - num_actual_predict_examples)
    tf.compat.v1.logging.info("  Batch size = %d", FLAGS.predict_batch_size)

    predict_input_fn = file_based_input_fn_builder(
        input_file=predict_file,
        seq_length=FLAGS.max_seq_length,
        is_training=False,
        drop_remainder=False)

    result = estimator.predict(input_fn=predict_input_fn)

    with tf.io.gfile.GFile(out_file, "w") as writer:
        num_written_lines = 0
        tf.compat.v1.logging.info("***** Predict results *****")
        for (i, prediction) in enumerate(result):
            predictions = prediction["predictions"]
            label_weights = prediction["label_weights"]
            if i >= num_actual_predict_examples:
                break
            output_line = "\n".join(label_list[tag]
                for k, tag in enumerate(predictions) if label_weights[k] > 0.) + "\n\n"
            writer.write(output_line)
            num_written_lines += 1
    assert num_written_lines == num_actual_predict_examples

In [230]:
estimator.predict()

TypeError: predict() missing 1 required positional argument: 'input_fn'

In [ ]:
import optimization

In [ ]:
pwd

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

predict_examples = processor._create_examples(processor.get_raw_data(test_file), "test")
# predict_examples = processor.get_test_examples(test_file)
num_actual_predict_examples = len(predict_examples)
predict_file = os.path.join(os.path.dirname(test_file), "predict.tf_record")
file_based_convert_examples_to_features(
  predict_examples, label_list, FLAGS.max_seq_length, tokenizer, predict_file)


predict_input_fn = file_based_input_fn_builder(
    input_file=predict_file,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

with tf.io.gfile.GFile(out_file, "w") as writer:
    num_written_lines = 0
    tf.compat.v1.logging.info("***** Predict results *****")
    for (i, prediction) in enumerate(result):
        print(1)
        predictions = prediction["predictions"]
        label_weights = prediction["label_weights"]
        if i >= num_actual_predict_examples:
            break
        output_line = "\n".join(label_list[tag]
            for k, tag in enumerate(predictions) if label_weights[k] > 0.) + "\n\n"
        writer.write(output_line)
        num_written_lines += 1
assert num_written_lines == num_actual_predict_examples

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

predict_examples = processor._create_examples(processor.get_raw_data(test_file), "test")
# predict_examples = processor.get_test_examples(test_file)
num_actual_predict_examples = len(predict_examples)
predict_file = os.path.join(os.path.dirname(test_file), "predict.tf_record")
file_based_convert_examples_to_features(
  predict_examples, label_list, FLAGS.max_seq_length, tokenizer, predict_file)


predict_input_fn = file_based_input_fn_builder(
    input_file=predict_file,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

with tf.io.gfile.GFile(out_file, "w") as writer:
    num_written_lines = 0
    tf.compat.v1.logging.info("***** Predict results *****")
    for (i, prediction) in enumerate(result):
        predictions = prediction["predictions"]
        label_weights = prediction["label_weights"]
        if i >= num_actual_predict_examples:
            break
        output_line = "\n".join(label_list[tag]
            for k, tag in enumerate(predictions) if label_weights[k] > 0.) + "\n\n"
        writer.write(output_line)
        num_written_lines += 1
assert num_written_lines == num_actual_predict_examples

In [ ]:
a = estimator.predict(input_fn=predict_input_fn)

In [ ]:
predict_input_fn = file_based_input_fn_builder(
    input_file=predict_file,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=False)

In [231]:
predict_examples = processor._create_examples(processor.get_raw_data(test_file), "test")
# predict_examples = processor.get_test_examples(test_file)
num_actual_predict_examples = len(predict_examples)
predict_file = os.path.join(os.path.dirname(test_file), "predict.tf_record")
file_based_convert_examples_to_features(
  predict_examples, label_list, FLAGS.max_seq_length, tokenizer, predict_file)


predict_input_fn = file_based_input_fn_builder(
    input_file=predict_file,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)
result


INFO:tensorflow:Writing example 0 of 2
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-0
INFO:tensorflow:tokens: [CLS] 美 纹 纸 真 的 让 人 又 爱 又 恨 ！ 真的 [SEP]
INFO:tensorflow:input_ids: 101 5400 5291 5290 4695 4637 6374 781 1347 4262 1347 2615 8012 21234 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

<generator object TPUEstimator.predict at 0x7f58e0570360>

In [ ]:
result.

In [232]:
num_written_lines = 0
tf.compat.v1.logging.info("***** Predict results *****")
for (i, prediction) in enumerate(result):
    predictions = prediction["predictions"]
    label_weights = prediction["label_weights"]
  


INFO:tensorflow:***** Predict results *****
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU/GPU
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ops/jupyter/语义/dataprocess/摘要-标题-图谱/内容理解/Lattice_Bert/tmp/best/model.ckpt-414
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:prediction_loop marked as finished
INFO:tensorflow:prediction_loop marked as finished


In [186]:
is_per_host

3

In [2]:
import tensorflow as tf

In [4]:
z = tf.compat.v1.estimator.tpu.RunConfig()
zz = tf.compat.v1.estimator.tpu.TPUConfig()

In [6]:
tf.compat.v1.distribute.MirroredStrategy

tensorflow.python.distribute.mirrored_strategy.MirroredStrategyV1

In [15]:
from tensorflow.python.distribute.cross_device_ops import AllReduceCrossDeviceOps

In [8]:
zz = tf.compat.v1.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [ ]:
AllReduceCrossDeviceOps()

In [16]:
tf.compat.v1.distribute.MirroredStrategy(
        devices=["/gpu:0","/gpu:1"],
        cross_device_ops=AllReduceCrossDeviceOps('nccl', num_packs=2),
    )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [1]:
import tensorflow as tf
aaa = tf.compat.v1.estimator.Estimator()

TypeError: __init__() missing 1 required positional argument: 'model_fn'

In [2]:
pwd

'/home/ops/jupyter/语义/dataprocess/摘要-标题-图谱/内容理解/Lattice_Bert/AliceMind_main/LatticeBERT'